In [1]:
import pandas as pd
pd.set_option('display.max_rows', 60)
pd.set_option('display.max_columns', None) 
import numpy as np
import re
from collections import defaultdict

In [2]:
ids = pd.read_csv('Raw/player_ids.csv')
ids.head()

,gsis_id,stats_id,mapped_ids,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,cleaned_name_yahoo,match_key_yahoo,last_name_yahoo
0,00-0016919,3727,3727,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,331.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2014,adam vinatieri,adam vinatieri_3727,vinatieri
1,00-0016919,3727,3727,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,348.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2015,adam vinatieri,adam vinatieri_3727,vinatieri
2,00-0016919,3727,3727,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,359.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2016,adam vinatieri,adam vinatieri_3727,vinatieri
3,00-0016919,3727,3727,11,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,371.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2017,adam vinatieri,adam vinatieri_3727,vinatieri
4,00-0016919,3727,3727,9,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,380.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2018,adam vinatieri,adam vinatieri_3727,vinatieri


In [3]:
ids[ids['cleaned_name_yahoo'] == 'travis hunter']

,gsis_id,stats_id,mapped_ids,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,cleaned_name_yahoo,match_key_yahoo,last_name_yahoo
11772,00-0040718,41787,99001,8,WR,nfl.p.41787,Jax,Jacksonville Jaguars,nfl.t.30,https://sports.yahoo.com/nfl/teams/jacksonville/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/VMwr0OLB8DWK...,https://s.yimg.com/iu/api/res/1.2/VMwr0OLB8DWK...,False,False,False,0,Travis,Travis,Travis Hunter,Hunter,99001,461.p.99001,O,WR,https://sports.yahoo.com/nfl/players/41787,2025,travis hunter,travis hunter_99001,hunter


In [4]:
ids = ids.rename(columns={'stats_id':'true_y_id'})

In [5]:
ids.head()

,gsis_id,true_y_id,mapped_ids,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,cleaned_name_yahoo,match_key_yahoo,last_name_yahoo
0,00-0016919,3727,3727,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,331.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2014,adam vinatieri,adam vinatieri_3727,vinatieri
1,00-0016919,3727,3727,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,348.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2015,adam vinatieri,adam vinatieri_3727,vinatieri
2,00-0016919,3727,3727,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,359.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2016,adam vinatieri,adam vinatieri_3727,vinatieri
3,00-0016919,3727,3727,11,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,371.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2017,adam vinatieri,adam vinatieri_3727,vinatieri
4,00-0016919,3727,3727,9,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,380.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2018,adam vinatieri,adam vinatieri_3727,vinatieri


In [6]:
ids.columns.duplicated().sum()

np.int64(0)

In [7]:
ids[ids['mapped_ids'] != ids['player_id']]

,gsis_id,true_y_id,mapped_ids,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,cleaned_name_yahoo,match_key_yahoo,last_name_yahoo


In [8]:
ids = ids[['gsis_id', 'cleaned_name_yahoo', 'true_y_id', 'editorial_team_key', 'editorial_team_abbr', 'player_id', 'player_key', 'primary_position', 'season', 'last_name_yahoo']]

In [9]:
ids['gs_unq'] = ids['gsis_id'] + '_' + ids['season'].astype(str)

In [10]:
ids.head()

,gsis_id,cleaned_name_yahoo,true_y_id,editorial_team_key,editorial_team_abbr,player_id,player_key,primary_position,season,last_name_yahoo,gs_unq
0,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,331.p.3727,K,2014,vinatieri,00-0016919_2014
1,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,348.p.3727,K,2015,vinatieri,00-0016919_2015
2,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,359.p.3727,K,2016,vinatieri,00-0016919_2016
3,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,371.p.3727,K,2017,vinatieri,00-0016919_2017
4,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,380.p.3727,K,2018,vinatieri,00-0016919_2018


In [11]:
y_stats = pd.read_csv('Raw/stats_yahooleague_raw_14to25.csv')
y_stats.head()

,Unnamed: 0,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,player_points.coverage_type,player_points.season,player_points.total,player_stats.coverage_type,player_stats.season,player_stats.stats,position_type,primary_position,url
0,0,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,331.p.3727,season,2014,153.00,season,2014,"[{'stat': {'stat_id': 19, 'value': 0.0}}, {'st...",K,K,https://sports.yahoo.com/nfl/players/3727
1,1,9,TE,nfl.p.3950,Atl,Atlanta Falcons,nfl.t.1,https://sports.yahoo.com/nfl/teams/atlanta/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,False,False,False,0,Tony,Tony,Tony Gonzalez,Gonzalez,3950,331.p.3950,season,2014,0.00,season,2014,"[{'stat': {'stat_id': 2, 'value': 0.0}}, {'sta...",O,TE,https://sports.yahoo.com/nfl/players/3950
2,2,4,QB,nfl.p.4256,Den,Denver Broncos,nfl.t.7,https://sports.yahoo.com/nfl/teams/denver/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,False,False,False,0,Peyton,Peyton,Peyton Manning,Manning,4256,331.p.4256,season,2014,398.23,season,2014,"[{'stat': {'stat_id': 2, 'value': 0.0}}, {'sta...",O,QB,https://sports.yahoo.com/nfl/players/4256
3,3,4,K,nfl.p.4269,Ari,Arizona Cardinals,nfl.t.22,https://sports.yahoo.com/nfl/teams/arizona/,['K'],small,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,False,False,False,0,Phil,Phil,Phil Dawson,Dawson,4269,331.p.4269,season,2014,125.00,season,2014,"[{'stat': {'stat_id': 19, 'value': 0.0}}, {'st...",K,K,https://sports.yahoo.com/nfl/players/4269
4,4,10,QB,nfl.p.4416,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,False,False,False,0,Matt,Matt,Matt Hasselbeck,Hasselbeck,4416,331.p.4416,season,2014,20.79,season,2014,"[{'stat': {'stat_id': 2, 'value': 0.0}}, {'sta...",O,QB,https://sports.yahoo.com/nfl/players/4416


In [12]:
y_stats = y_stats.drop(columns='Unnamed: 0')
y_stats.head()

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,player_points.coverage_type,player_points.season,player_points.total,player_stats.coverage_type,player_stats.season,player_stats.stats,position_type,primary_position,url
0,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,331.p.3727,season,2014,153.00,season,2014,"[{'stat': {'stat_id': 19, 'value': 0.0}}, {'st...",K,K,https://sports.yahoo.com/nfl/players/3727
1,9,TE,nfl.p.3950,Atl,Atlanta Falcons,nfl.t.1,https://sports.yahoo.com/nfl/teams/atlanta/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,False,False,False,0,Tony,Tony,Tony Gonzalez,Gonzalez,3950,331.p.3950,season,2014,0.00,season,2014,"[{'stat': {'stat_id': 2, 'value': 0.0}}, {'sta...",O,TE,https://sports.yahoo.com/nfl/players/3950
2,4,QB,nfl.p.4256,Den,Denver Broncos,nfl.t.7,https://sports.yahoo.com/nfl/teams/denver/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,False,False,False,0,Peyton,Peyton,Peyton Manning,Manning,4256,331.p.4256,season,2014,398.23,season,2014,"[{'stat': {'stat_id': 2, 'value': 0.0}}, {'sta...",O,QB,https://sports.yahoo.com/nfl/players/4256
3,4,K,nfl.p.4269,Ari,Arizona Cardinals,nfl.t.22,https://sports.yahoo.com/nfl/teams/arizona/,['K'],small,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,False,False,False,0,Phil,Phil,Phil Dawson,Dawson,4269,331.p.4269,season,2014,125.00,season,2014,"[{'stat': {'stat_id': 19, 'value': 0.0}}, {'st...",K,K,https://sports.yahoo.com/nfl/players/4269
4,10,QB,nfl.p.4416,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,False,False,False,0,Matt,Matt,Matt Hasselbeck,Hasselbeck,4416,331.p.4416,season,2014,20.79,season,2014,"[{'stat': {'stat_id': 2, 'value': 0.0}}, {'sta...",O,QB,https://sports.yahoo.com/nfl/players/4416


In [13]:
y_stats = y_stats[['bye_weeks.week', 'player_key', 'player_points.total']]

In [14]:
y_stats['player_key'].nunique()

14096

In [15]:
ids['player_key'].nunique()

12157

In [16]:
ids['player_key'].isin(y_stats['player_key']).sum()

np.int64(12157)

In [17]:
y_stats[~(y_stats['player_key'].isin(ids['player_key']))]

,bye_weeks.week,player_key,player_points.total
107,9,331.p.8054,1.8
348,12,331.p.24746,0.0
385,11,331.p.24898,0.0
423,4,331.p.25050,0.0
432,5,331.p.25123,0.0
...,...,...,...
14057,11,461.p.42587,0.0
14058,8,461.p.42592,0.0
14059,14,461.p.42596,0.0
14060,11,461.p.42597,0.0


In [18]:
ids_w_points = ids.merge(y_stats, how='inner', on='player_key')
ids_w_points

,gsis_id,cleaned_name_yahoo,true_y_id,editorial_team_key,editorial_team_abbr,player_id,player_key,primary_position,season,last_name_yahoo,gs_unq,bye_weeks.week,player_points.total
0,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,331.p.3727,K,2014,vinatieri,00-0016919_2014,10,153.00
1,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,348.p.3727,K,2015,vinatieri,00-0016919_2015,10,119.00
2,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,359.p.3727,K,2016,vinatieri,00-0016919_2016,10,149.00
3,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,371.p.3727,K,2017,vinatieri,00-0016919_2017,11,123.00
4,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,380.p.3727,K,2018,vinatieri,00-0016919_2018,9,123.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12152,Houston Texans,houston texans,100034,nfl.t.34,Hou,100034,406.p.100034,DEF,2021,texans,Houston Texans_2021,10,156.38
12153,Houston Texans,houston texans,100034,nfl.t.34,Hou,100034,414.p.100034,DEF,2022,texans,Houston Texans_2022,6,171.25
12154,Houston Texans,houston texans,100034,nfl.t.34,Hou,100034,423.p.100034,DEF,2023,texans,Houston Texans_2023,7,205.00
12155,Houston Texans,houston texans,100034,nfl.t.34,Hou,100034,449.p.100034,DEF,2024,texans,Houston Texans_2024,14,242.00


In [19]:
ids_w_points.columns.duplicated().sum()

np.int64(0)

In [20]:
ids_w_points.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12157 entries, 0 to 12156
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gsis_id              12157 non-null  object 
 1   cleaned_name_yahoo   12157 non-null  object 
 2   true_y_id            12157 non-null  int64  
 3   editorial_team_key   12157 non-null  object 
 4   editorial_team_abbr  12157 non-null  object 
 5   player_id            12157 non-null  int64  
 6   player_key           12157 non-null  object 
 7   primary_position     12157 non-null  object 
 8   season               12157 non-null  int64  
 9   last_name_yahoo      12157 non-null  object 
 10  gs_unq               12157 non-null  object 
 11  bye_weeks.week       12157 non-null  int64  
 12  player_points.total  12157 non-null  float64
dtypes: float64(1), int64(4), object(8)
memory usage: 1.2+ MB


In [21]:
nfl_player_stats = pd.read_csv('Raw/stats_player_nfl_raw_14to25.csv')
nfl_player_stats.head()

,Unnamed: 0,player_id,player_name,player_display_name,position,position_group,headshot_url,season,season_type,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list,fantasy_points,fantasy_points_ppr
0,0,00-0004091,P.Dawson,Phil Dawson,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2014,REG,SF,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,1,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,31,5,1,55.0,0.806452,0,6,8,5,6,0,0,0,1,1,2,1,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948,250,45,33,33,0,0,1.0,1,1,0,0,35,0.00,0.00
1,1,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,REG,IND,4,30,44,301,2,0,2,-7,1,1,313,182,17,-0.628051,3.970076,0,0.961661,8,-11,0,0,0,0,-7.756981,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,16.94,16.94
2,2,00-0010346,P.Manning,Peyton Manning,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,REG,DEN,16,395,597,4727,39,15,17,-118,2,1,5071,1882,225,121.816891,3.316656,2,0.932163,24,-24,0,3,1,2,-24.812411,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,2,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,1,3,0,0,0,3,20,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,312.68,312.68
3,3,00-0016919,A.Vinatieri,Adam Vinatieri,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2014,REG,IND,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,31,1,0,53.0,0.967742,0,10,10,7,3,0,0,0,0,1,0,0,25;46;27;48;43;25;31;30;38;34;27;34;23;50;31;2...,46,NaN,1049,46,0,50,50,0,0,1.0,0,0,0,0,NaN,0.00,0.00
4,4,00-0018227,C.Woodson,Charles Woodson,FS,DB,https://static.www.nfl.com/image/private/f_aut...,2014,REG,LV,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,75,8,30,4,21,0,1.0,10.0,1,4,35,8,0,0,0,0,0,0,1,1,0,0,0,3,28,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,0.00,0.00


In [22]:
nfl_player_stats = nfl_player_stats.drop(columns='Unnamed: 0')
nfl_player_stats.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,season,season_type,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list,fantasy_points,fantasy_points_ppr
0,00-0004091,P.Dawson,Phil Dawson,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2014,REG,SF,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,1,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,31,5,1,55.0,0.806452,0,6,8,5,6,0,0,0,1,1,2,1,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948,250,45,33,33,0,0,1.0,1,1,0,0,35,0.00,0.00
1,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,REG,IND,4,30,44,301,2,0,2,-7,1,1,313,182,17,-0.628051,3.970076,0,0.961661,8,-11,0,0,0,0,-7.756981,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,16.94,16.94
2,00-0010346,P.Manning,Peyton Manning,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,REG,DEN,16,395,597,4727,39,15,17,-118,2,1,5071,1882,225,121.816891,3.316656,2,0.932163,24,-24,0,3,1,2,-24.812411,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,2,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,1,3,0,0,0,3,20,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,312.68,312.68
3,00-0016919,A.Vinatieri,Adam Vinatieri,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2014,REG,IND,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,31,1,0,53.0,0.967742,0,10,10,7,3,0,0,0,0,1,0,0,25;46;27;48;43;25;31;30;38;34;27;34;23;50;31;2...,46,NaN,1049,46,0,50,50,0,0,1.0,0,0,0,0,NaN,0.00,0.00
4,00-0018227,C.Woodson,Charles Woodson,FS,DB,https://static.www.nfl.com/image/private/f_aut...,2014,REG,LV,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,75,8,30,4,21,0,1.0,10.0,1,4,35,8,0,0,0,0,0,0,1,1,0,0,0,3,28,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,0.00,0.00


In [23]:
nfl_player_stats['recent_team'].value_counts()

recent_team
DET    758
NYJ    755
WAS    754
NYG    754
HOU    743
TEN    739
SF     737
CLE    737
ARI    732
NO     729
CAR    725
BAL    725
IND    723
LAC    721
MIA    716
CHI    715
JAX    715
BUF    713
SEA    711
PHI    710
LV     709
NE     708
DAL    705
TB     702
DEN    700
ATL    689
CIN    687
LA     682
KC     680
PIT    673
GB     671
MIN    670
Name: count, dtype: int64

In [24]:
nfl_player_stats['recent_team'].nunique()

32

In [25]:
nfl_player_stats[nfl_player_stats['player_display_name'] == 'Phil Dawson']

,player_id,player_name,player_display_name,position,position_group,headshot_url,season,season_type,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list,fantasy_points,fantasy_points_ppr
0,00-0004091,P.Dawson,Phil Dawson,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2014,REG,SF,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,1,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,31,5,1,55.0,0.806452,0,6,8,5,6,0,0,0,1,1,2,1,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948,250,45,33,33,0,0,1.000000,1,1,0,0,35,0.0,0.0
1850,00-0004091,P.Dawson,Phil Dawson,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2015,REG,SF,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,27,1,2,54.0,0.888889,0,9,6,6,3,0,0,0,0,1,0,0,30;25;47;33;43;22;53;31;26;42;35;54;26;44;27;2...,45,28;41,848,45,69,20,21,0,1,0.952381,1,1,0,0,23,0.0,0.0
3696,00-0004091,P.Dawson,Phil Dawson,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2016,REG,SF,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,18,21,3,0,53.0,0.857143,0,4,6,7,1,0,0,0,0,2,1,0,29;39;22;36;33;48;47;28;49;48;46;49;53;33;36;3...,53;44;48,NaN,702,145,0,33,34,1,0,0.970588,0,0,0,0,NaN,0.0,0.0
5552,00-0004091,P.Dawson,Phil Dawson,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2017,REG,ARI,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,1,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,32,40,6,2,57.0,0.800000,1,7,10,10,4,0,0,0,4,2,0,0,24;27;40;30;37;29;43;50;32;28;25;32;43;34;42;4...,32;42;36;32;38;40,51;45,1183,220,96,23,26,1,2,0.884615,1,1,0,0,30,0.0,0.0
7421,00-0004091,P.Dawson,Phil Dawson,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2018,REG,ARI,10,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,8,3,0,43.0,0.625000,0,2,2,1,0,0,0,0,0,2,1,0,23;26;43;31;30,50;45;46,NaN,153,141,0,15,15,0,0,1.000000,0,0,0,0,NaN,0.0,0.0


In [26]:
nfl_player_stats['position'].value_counts()

position
WR     2685
LB     2125
CB     2026
DE     1795
RB     1785
DT     1570
TE     1501
OT     1259
G      1152
DB     1063
QB      904
OLB     721
SAF     617
C       581
K       495
S       468
P       435
FS      423
ILB     334
LS      332
MLB     218
FB      188
NT      154
DL       35
OL       11
Name: count, dtype: int64

In [27]:
nfl_player_stats = nfl_player_stats.loc[nfl_player_stats['position'].isin(['QB', 'WR', 'RB', 'TE', 'K'])]

In [28]:
nfl_player_stats['gs_unq'] = nfl_player_stats['player_id'] + '_' + nfl_player_stats['season'].astype(str)
nfl_player_stats.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,season,season_type,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list,fantasy_points,fantasy_points_ppr,gs_unq
0,00-0004091,P.Dawson,Phil Dawson,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2014,REG,SF,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,1,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,31,5,1,55.0,0.806452,0,6,8,5,6,0,0,0,1,1,2,1,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948,250,45,33,33,0,0,1.000000,1,1,0,0,35,0.00,0.00,00-0004091_2014
1,00-0007091,M.Hasselbeck,Matt Hasselbeck,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,REG,IND,4,30,44,301,2,0,2,-7,1,1,313,182,17,-0.628051,3.970076,0,0.961661,8,-11,0,0,0,0,-7.756981,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,16.94,16.94,00-0007091_2014
2,00-0010346,P.Manning,Peyton Manning,QB,QB,https://static.www.nfl.com/image/private/f_aut...,2014,REG,DEN,16,395,597,4727,39,15,17,-118,2,1,5071,1882,225,121.816891,3.316656,2,0.932163,24,-24,0,3,1,2,-24.812411,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,2,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,1,3,0,0,0,3,20,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,312.68,312.68,00-0010346_2014
3,00-0016919,A.Vinatieri,Adam Vinatieri,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2014,REG,IND,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,31,1,0,53.0,0.967742,0,10,10,7,3,0,0,0,0,1,0,0,25;46;27;48;43;25;31;30;38;34;27;34;23;50;31;2...,46,NaN,1049,46,0,50,50,0,0,1.000000,0,0,0,0,NaN,0.00,0.00,00-0016919_2014
6,00-0019536,S.Graham,Shayne Graham,K,SPEC,https://static.www.nfl.com/image/private/f_aut...,2014,REG,NO,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19,22,3,0,50.0,0.863636,0,6,9,3,1,0,0,0,0,2,1,0,31;50;27;30;30;29;44;27;48;36;31;29;37;40;31;2...,41;42;51,NaN,636,134,0,46,47,0,1,0.978723,0,0,0,0,NaN,0.00,0.00,00-0019536_2014


In [29]:
nfl_player_stats = nfl_player_stats.drop(columns=['player_id', 'player_name', 'player_display_name', 'position', 'position_group', 'headshot_url', 'season', 'season_type', 'fantasy_points', 'fantasy_points_ppr'])

In [30]:
nfl_player_stats.head()

,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list,gs_unq
0,SF,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,1,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,31,5,1,55.0,0.806452,0,6,8,5,6,0,0,0,1,1,2,1,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948,250,45,33,33,0,0,1.000000,1,1,0,0,35,00-0004091_2014
1,IND,4,30,44,301,2,0,2,-7,1,1,313,182,17,-0.628051,3.970076,0,0.961661,8,-11,0,0,0,0,-7.756981,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,00-0007091_2014
2,DEN,16,395,597,4727,39,15,17,-118,2,1,5071,1882,225,121.816891,3.316656,2,0.932163,24,-24,0,3,1,2,-24.812411,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,2,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,1,3,0,0,0,3,20,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,00-0010346_2014
3,IND,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,31,1,0,53.0,0.967742,0,10,10,7,3,0,0,0,0,1,0,0,25;46;27;48;43;25;31;30;38;34;27;34;23;50;31;2...,46,NaN,1049,46,0,50,50,0,0,1.000000,0,0,0,0,NaN,00-0016919_2014
6,NO,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.0,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19,22,3,0,50.0,0.863636,0,6,9,3,1,0,0,0,0,2,1,0,31;50;27;30;30;29;44;27;48;36;31;29;37;40;31;2...,41;42;51,NaN,636,134,0,46,47,0,1,0.978723,0,0,0,0,NaN,00-0019536_2014


In [31]:
nfl_player_stats[nfl_player_stats['gs_unq'].isin(ids_w_points['gs_unq'])]

,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list,gs_unq
0,SF,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.000000,0.000000,0.000000,0,1,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,31,5,1,55.0,0.806452,0,6,8,5,6,0,0,0,1,1,2,1,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948,250,45,33,33,0,0,1.000000,1,1,0,0,35,00-0004091_2014
1,IND,4,30,44,301,2,0,2,-7,1,1,313,182,17,-0.628051,3.970076,0,0.961661,8,-11,0,0,0,0,-7.756981,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,00-0007091_2014
2,DEN,16,395,597,4727,39,15,17,-118,2,1,5071,1882,225,121.816891,3.316656,2,0.932163,24,-24,0,3,1,2,-24.812411,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.000000,0.000000,0.000000,0,2,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,1,3,0,0,0,3,20,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,00-0010346_2014
3,IND,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,31,1,0,53.0,0.967742,0,10,10,7,3,0,0,0,0,1,0,0,25;46;27;48;43;25;31;30;38;34;27;34;23;50;31;2...,46,NaN,1049,46,0,50,50,0,0,1.000000,0,0,0,0,NaN,00-0016919_2014
6,NO,16,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19,22,3,0,50.0,0.863636,0,6,9,3,1,0,0,0,0,2,1,0,31;50;27;30;30;29;44;27;48;36;31;29;37;40;31;2...,41;42;51,NaN,636,134,0,46,47,0,1,0.978723,0,0,0,0,NaN,00-0019536_2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22871,LA,3,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,NaN,1,0,0,0,0,0,-1.549391,0,2,4,52,1,0,0,103,3,2,5.562502,0,0.504854,0.016807,0.048631,0.059252,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,0,0,0,NaN,00-0040737_2025
22873,SEA,7,0,0,0,0,0,0,0,0,0,0,0,

In [32]:
ids_w_points.loc[(~ids_w_points['gs_unq'].isin(nfl_player_stats['gs_unq']) & (ids_w_points['player_points.total'] <= 0))]

,gsis_id,cleaned_name_yahoo,true_y_id,editorial_team_key,editorial_team_abbr,player_id,player_key,primary_position,season,last_name_yahoo,gs_unq,bye_weeks.week,player_points.total
6,00-0016919,adam vinatieri,3727,nfl.t.11,Ind,3727,399.p.3727,K,2020,vinatieri,00-0016919_2020,7,0.0
7,00-0006101,tony gonzalez,3950,nfl.t.1,Atl,3950,331.p.3950,TE,2014,gonzalez,00-0006101_2014,9,0.0
15,00-0004091,phil dawson,4269,nfl.t.22,Ari,4269,390.p.4269,K,2019,dawson,00-0004091_2019,12,0.0
18,00-0000108,david akers,4587,nfl.t.8,Det,4587,331.p.4587,K,2014,akers,00-0000108_2014,9,0.0
22,00-0019646,sebastian janikowski,5046,nfl.t.26,Sea,5046,371.p.5046,K,2017,janikowski,00-0019646_2017,6,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11760,00-0040556,corey kiner,42386,nfl.t.16,Min,42386,461.p.42386,RB,2025,kiner,00-0040556_2025,6,0.0
11766,00-0040024,darius cooper,42464,nfl.t.21,Phi,42464,461.p.42464,WR,2025,cooper,00-0040024_2025,9,0.0
11767,00-0040315,jacorey brooks,42482,nfl.t.28,Was,42482,461.p.42482,WR,2025,brooks,00-0040315_2025,12,0.0
11768,00-0040617,luke floriea,42507,nfl.t.5,Cle,42507,461.p.42507,WR,2025,floriea,00-0040617_2025,9,0.0


In [33]:
nfl_player_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7370 entries, 0 to 22887
Columns: 104 entries, recent_team to gs_unq
dtypes: float64(14), int64(84), object(6)
memory usage: 5.9+ MB


In [34]:
ids_w_ply_stats = ids_w_points.merge(nfl_player_stats, how ='outer', on='gs_unq')
ids_w_ply_stats.head(10)

,gsis_id,cleaned_name_yahoo,true_y_id,editorial_team_key,editorial_team_abbr,player_id,player_key,primary_position,season,last_name_yahoo,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list
0,00-0000108,david akers,4587.0,nfl.t.8,Det,4587.0,331.p.4587,K,2014.0,akers,00-0000108_2014,9.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00-0004091,phil dawson,4269.0,nfl.t.22,Ari,4269.0,331.p.4269,K,2014.0,dawson,00-0004091_2014,4.0,125.00,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,31.0,5.0,1.0,55.0,0.806452,0.0,6.0,8.0,5.0,6.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948.0,250.0,45.0,33.0,33.0,0.0,0.0,1.000000,1.0,1.0,0.0,0.0,35
2,00-0004091,phil dawson,4269.0,nfl.t.22,Ari,4269.0,348.p.4269,K,2015.0,dawson,00-0004091_2015,9.0,102.00,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,27.0,1.0,2.0,54.0,0.888889,0.0,9.0,6.0,6.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,30;25;47;33;43;22;53;31;26;42;35;54;26;44;27;2...,45,28;41,848.0,45.0,69.0,20.0,21.0,0.0,1.0,0.952381,1.0,1.0,0.0,0.0,23
3,00-0004091,phil dawson,4269.0,nfl.t.22,Ari,4269.0,359.p.4269,K,2016.0,dawson,00-0004091_2016,9.0,95.00,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,21.0,3.0,0.0,53.0,0.857143,0.0,4.0,6.0,7.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,29;39;22;36;33;48;47;28;49;48;46;49;53;33;36;3...,53;44;48,NaN,702.0,145.0,0.0,33.0,34.0,1.0,0.0,0.970588,0.0,0

In [35]:
ids_w_ply_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12191 entries, 0 to 12190
Columns: 116 entries, gsis_id to gwfg_distance_list
dtypes: float64(103), object(13)
memory usage: 10.8+ MB


In [36]:
indv_stats = ids_w_ply_stats.loc[~((ids_w_ply_stats['recent_team'].isna()) & (ids_w_ply_stats['games'].isna())) | (ids_w_ply_stats['primary_position'] == 'DEF')]

In [37]:
indv_stats.columns

Index(['gsis_id', 'cleaned_name_yahoo', 'true_y_id', 'editorial_team_key',
       'editorial_team_abbr', 'player_id', 'player_key', 'primary_position',
       'season', 'last_name_yahoo',
       ...
       'pat_made', 'pat_att', 'pat_missed', 'pat_blocked', 'pat_pct',
       'gwfg_made', 'gwfg_att', 'gwfg_missed', 'gwfg_blocked',
       'gwfg_distance_list'],
      dtype='object', length=116)

In [39]:
ids[ids['gsis_id'] == '00-0023968']

,gsis_id,cleaned_name_yahoo,true_y_id,editorial_team_key,editorial_team_abbr,player_id,player_key,primary_position,season,last_name_yahoo,gs_unq


In [41]:
indv_stats = indv_stats.dropna(subset='cleaned_name_yahoo')
indv_stats.head()

,gsis_id,cleaned_name_yahoo,true_y_id,editorial_team_key,editorial_team_abbr,player_id,player_key,primary_position,season,last_name_yahoo,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list
1,00-0004091,phil dawson,4269.0,nfl.t.22,Ari,4269.0,331.p.4269,K,2014.0,dawson,00-0004091_2014,4.0,125.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,31.0,5.0,1.0,55.0,0.806452,0.0,6.0,8.0,5.0,6.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,27;24;29;51;46;31;31;55;52;27;30;54;22;34;45;3...,37;51;55;47;60,45,948.0,250.0,45.0,33.0,33.0,0.0,0.0,1.000000,1.0,1.0,0.0,0.0,35
2,00-0004091,phil dawson,4269.0,nfl.t.22,Ari,4269.0,348.p.4269,K,2015.0,dawson,00-0004091_2015,9.0,102.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,27.0,1.0,2.0,54.0,0.888889,0.0,9.0,6.0,6.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,30;25;47;33;43;22;53;31;26;42;35;54;26;44;27;2...,45,28;41,848.0,45.0,69.0,20.0,21.0,0.0,1.0,0.952381,1.0,1.0,0.0,0.0,23
3,00-0004091,phil dawson,4269.0,nfl.t.22,Ari,4269.0,359.p.4269,K,2016.0,dawson,00-0004091_2016,9.0,95.0,SF,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,21.0,3.0,0.0,53.0,0.857143,0.0,4.0,6.0,7.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,29;39;22;36;33;48;47;28;49;48;46;49;53;33;36;3...,53;44;48,NaN,702.0,145.0,0.0,33.0,34.0,1.0,0.0,0.970588,0.0,0.0,0.0,0.0,NaN
4,00-0004091,phil dawson,4269.0,nfl.t.22,Ari,4269.0,371.p.4269,K,2017.0,dawson,00-0004091_2017,8.0,137.0,ARI,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,40.0,6.0,2.0,57.0,0.800000,1.0,7.0,10.0,10.0,4.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,24;27;40;30;37;29;43;50;32;28;25;32;43

In [43]:
indv_stats[indv_stats['primary_position'] == 'DEF']

,gsis_id,cleaned_name_yahoo,true_y_id,editorial_team_key,editorial_team_abbr,player_id,player_key,primary_position,season,last_name_yahoo,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list
11805,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,331.p.100022,DEF,2014.0,cardinals,Arizona Cardinals_2014,4.0,213.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11806,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,348.p.100022,DEF,2015.0,cardinals,Arizona Cardinals_2015,9.0,253.68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11807,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,359.p.100022,DEF,2016.0,cardinals,Arizona Cardinals_2016,9.0,235.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11808,Arizona Cardinals,arizona cardinals,100022.0,nfl.t.22,Ari,100022.0,371.p.100022,DEF,2017.0,cardinals,Arizona Cardinals_2017,8.0,216.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11809,Arizona Cardinals,arizona cardinals,100022.0

In [47]:
indv_stats[(indv_stats['def_tackles_for_loss'].notna()) & (indv_stats['def_tackles_for_loss'] > 0)]

,gsis_id,cleaned_name_yahoo,true_y_id,editorial_team_key,editorial_team_abbr,player_id,player_key,primary_position,season,last_name_yahoo,gs_unq,bye_weeks.week,player_points.total,recent_team,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,pacr,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list
1882,00-0029051,jarrett boykin,26083.0,nfl.t.9,GB,26083.0,331.p.26083,WR,2014.0,boykin,00-0029051_2014,9.0,5.30,GB,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,3.0,12.0,23.0,0.0,0.0,0.0,133.0,27.0,1.0,-9.838036,0.0,0.172932,0.022599,0.030751,0.055424,0.0,2.0,0.0,1.0,1.0,12.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN
1975,00-0029239,brandon bolden,26389.0,nfl.t.13,LV,26389.0,348.p.26389,RB,2015.0,bolden,00-0029239_2015,6.0,69.70,NE,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,63.0,207.0,0.0,0.0,0.0,9.0,-16.403109,0.0,19.0,30.0,180.0,2.0,0.0,0.0,65.0,129.0,8.0,2.242350,0.0,2.769231,0.048077,0.013171,0.081335,0.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN
4375,00-0031697,jordan taylor,28727.0,nfl.t.7,Den,28727.0,371.p.28727,WR,2017.0,taylor,00-0031697_2017,5.0,27.20,DEN,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,13.0,20.0,142.0,0.0,0.0,0.0,206.0,45.0,5.0,-0.047186,0.0,0.689320,0.036298,0.043378,0.084811,0.0,4.0,0.0,0.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,11.0,103.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN
6031,00-0033251,zach pascal,30502.0,nfl.t.19,NYG,30502.0,390.p.30502,WR,2019.0,pascal,00-0033251_2019,11.0,137.20,IND,16.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,-0.704225,-42.273840,0.0,0.000000,2.0,16.0,0.0,0.0,0.0,1.0,0.907318,0.0,41.0,72.0,607.0,5.0,0.0,0.0,810.0,239.0,29.0,20.319164,1.0,0.749383,0.147541,0.196364,0.358766,0.0,6.0,0.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,20.0,0.0,0.0,10.0,263.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,

In [54]:
indv_stats = indv_stats.drop(columns=(indv_stats.columns[indv_stats.columns.str.startswith('def_')].tolist()))

In [55]:
indv_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7720 entries, 1 to 12190
Columns: 101 entries, gsis_id to gwfg_distance_list
dtypes: float64(88), object(13)
memory usage: 6.0+ MB
